# Pré-processamento de Texto em Python

Existem diversas etapas diferentes envolvendo o pré-processamento de um texto em python. A aplicação em que será empregado pode gerar variações na ordem destas etapas, bem como possibilitar que algumas sejam omitidas. Neste _notebook_ exploraremos uma alternativa para cumprir a tarefa de forma mais genérica.

1. [Remoção de tags HTML](#Remo%C3%A7%C3%A3o-de-tags-HTML)
1. [Tratamento-para-Acentuação](#Tratamento-para-Acentua%C3%A7%C3%A3o)
1. [Tokenização](#Tokeniza%C3%A7%C3%A3o)
1. Remoção de _Stop Words_
1. 
2. Lemmatização
3. Outras etapas

[Referências](#Refer%C3%AAncias)

## Remoção de tags HTML

Muitos textos provém de _crawlers_ que acabam retornando muitos "lixos" junto com o material de interesse. Um destes "lixos" são tags HTML.

Remove apenas as tags HTML (incluindo parâmetros), mantendo o texto contido nelas:

In [3]:
from bs4 import BeautifulSoup


def strip_html_tags(text, separator=""):
    parser = BeatifulSoup(text, "html.parser")
    
    return parser.get_text(separator=separator)

Remove as tags e o conteúdo:

In [4]:
import re


def delete_html_nodes(text):
    regex = re.compile("<.+>")
    
    return re.sub(regex, "", text)

## Tratamento para Acentuação

Caracteres acentuados como `à`, `ä` e `é` introduzem distinções (por exemplo `a` $\neq$ `à` $\neq$ `á`) indesejadas, que em etapas posteriores ao pré-processamento resultarão em problemas maiores.

Para evitar isto, basta remover estas acentuações. Uma forma simples de fazê-lo é converter caracteres no formato unicode para o formato ascii.

In [ ]:
import unidecode

def utf8_to_ascii(text):
    return unidecode.unidecode(text)

## Tokenização

[explicar]

In [11]:
import spacy
spacy.cli.download("en_core_web_md")

def tokenize(corpus):
    nlp = spacy.load("en_core_web_md")
    
    tokens = []
    for document in corpus:
        spacy_doc = nlp(document)
        
        tokens.append([token for token in spacy_doc])
            
    return tokens


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


## Lematização e _Stemming_

Os processos de lematização e de _stemming_ possuem o mesmo objetivo: reduzir formas flexionadas de linguagem (por exemplo verbos conjugados) para seu formato elementar. A diferença básica entre as técnicas é, sem muito rigor, que a lematização é um processo mais inteligente, capaz de encontrar os radicais mesmo para verbos irregulares (para o caso em que a palavra analisada é um verbo :)). Em geral, utilizar a lematização elimina a necessidade de _stemming_, e portanto utilizaremos apenas ela. 

## Tratamento para Números

[explicar]



## Referências

[1] [All you need to know about text preprocessing for NLP and Machine Learning](https://www.kdnuggets.com/2019/04/text-preprocessing-nlp-machine-learning.html);

[2] [NLP Text Preprocessing: A Practical Guide and Template](https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79);

[3] []();

[4] []();